<a href="https://colab.research.google.com/github/YugantGotmare/Machine-Learning-Projects/blob/master/Paramount_TV_Shows_and_Movies_Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [27]:
movies_data = pd.read_csv('/content/titles.csv')

In [28]:
movies_data.head()

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score,index
0,tm19248,The General,MOVIE,"During America’s Civil War, Union spies steal ...",1926,NaN,78,"['action', 'drama', 'war', 'western', 'comedy'...",['US'],NaN,tt0017925,8.2,89766.0,8.647,8.0,1
1,tm83884,His Girl Friday,MOVIE,"Hildy, the journalist former wife of newspaper...",1940,NaN,92,"['comedy', 'drama', 'romance']",['US'],NaN,tt0032599,7.8,57835.0,11.270,7.4,2
2,tm19424,Detour,MOVIE,"The life of Al Roberts, a pianist in a New Yor...",1945,NaN,66,"['thriller', 'drama', 'crime']",['US'],NaN,tt0037638,7.3,17233.0,7.757,7.2,3
3,tm112005,Marihuana,MOVIE,A young girl named Burma attends a beach party...,1936,NaN,57,"['crime', 'drama']",['US'],NaN,tt0026683,4.0,864.0,3.748,3.6,4
4,tm22806,Intolerance: Love's Struggle Throughout the Ages,MOVIE,"The story of a poor young woman, separated by ...",1916,NaN,197,"['history', 'drama']",['US'],NaN,tt0006864,7.7,15242.0,9.412,7.2,5


In [29]:
movies_data.tail()

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score,index
2820,ts305329,House Calls with Dr. Phil,SHOW,The talk show legend travels across the countr...,2021,TV-14,43,['reality'],['US'],1.0,tt15176234,5.4,35.0,0.600,NaN,2821
2821,ts291097,Disrupt & Dismantle with Soledad O'Brien,SHOW,NaN,2021,TV-MA,40,['documentation'],[],1.0,tt14315936,3.5,18.0,NaN,NaN,2822
2822,tm1100359,Race Against Time: The CIA and 9/11,MOVIE,Follows former top officials and elite operati...,2021,NaN,89,['documentation'],[],NaN,tt15392066,4.6,64.0,2.403,NaN,2823
2823,tm1097570,Virus Hunting: Cave to COVID,MOVIE,"As the world battles COVID-19, a team of exper...",2021,NaN,60,[],[],NaN,NaN,NaN,NaN,0.600,NaN,2824
2824,tm1032011,Yukon's Wild Grizzlies,MOVIE,"Deep in Canada's remote Yukon, a dramatic comi...",2021,G,44,['documentation'],[],NaN,NaN,NaN,NaN,0.600,7.0,2825


In [30]:
movies_data.shape

(2825, 16)

In [31]:
movies_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2825 entries, 0 to 2824
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    2825 non-null   object 
 1   title                 2825 non-null   object 
 2   type                  2825 non-null   object 
 3   description           2812 non-null   object 
 4   release_year          2825 non-null   int64  
 5   age_certification     1302 non-null   object 
 6   runtime               2825 non-null   int64  
 7   genres                2825 non-null   object 
 8   production_countries  2825 non-null   object 
 9   seasons               626 non-null    float64
 10  imdb_id               2666 non-null   object 
 11  imdb_score            2613 non-null   float64
 12  imdb_votes            2613 non-null   float64
 13  tmdb_popularity       2768 non-null   float64
 14  tmdb_score            2337 non-null   float64
 15  index                

In [32]:
movies_features = ["title", "description", "genres"]
print(movies_features)

['title', 'description', 'genres']


In [33]:
for features in movies_features:
  movies_data[features] = movies_data[features].fillna('')

In [34]:
combining_movies_features = movies_data['title']+' '+movies_data['description']+' '+movies_data['genres']
print(combining_movies_features)


0       The General During America’s Civil War, Union ...
1       His Girl Friday Hildy, the journalist former w...
2       Detour The life of Al Roberts, a pianist in a ...
3       Marihuana A young girl named Burma attends a b...
4       Intolerance: Love's Struggle Throughout the Ag...
                              ...                        
2820    House Calls with Dr. Phil The talk show legend...
2821    Disrupt & Dismantle with Soledad O'Brien  ['do...
2822    Race Against Time: The CIA and 9/11 Follows fo...
2823    Virus Hunting: Cave to COVID As the world batt...
2824    Yukon's Wild Grizzlies Deep in Canada's remote...
Length: 2825, dtype: object


In [35]:
vectorizer = TfidfVectorizer()

In [36]:
feature_vectors = vectorizer.fit_transform(combining_movies_features)
print(feature_vectors)

  (0, 5164)	0.1039503080831007
  (0, 3096)	0.049776179864791395
  (0, 16084)	0.08493035197349479
  (0, 4557)	0.04806412299336251
  (0, 405)	0.06433301137057178
  (0, 841)	0.20248598052119762
  (0, 12265)	0.12961150981647612
  (0, 1298)	0.09825579795086913
  (0, 6237)	0.09513362933178098
  (0, 1944)	0.11945526142178953
  (0, 14900)	0.06694626718036295
  (0, 11281)	0.12779338668642096
  (0, 7045)	0.05172169130726859
  (0, 7468)	0.03856627912336562
  (0, 665)	0.0831631809597085
  (0, 4424)	0.11229369633931408
  (0, 9893)	0.09528841492512999
  (0, 6696)	0.20248598052119762
  (0, 13427)	0.13370992037163584
  (0, 6847)	0.06342666003584331
  (0, 801)	0.06443696508646393
  (0, 1968)	0.20248598052119762
  (0, 1158)	0.18000622125832177
  (0, 779)	0.05747383428015648
  (0, 305)	0.1547461200421132
  :	:
  (2824, 8532)	0.09245156881327704
  (2824, 12181)	0.09730419300138327
  (2824, 6783)	0.10037666253445984
  (2824, 14091)	0.062053351864478155
  (2824, 10598)	0.07193339984011062
  (2824, 13630)	0.

In [37]:
similarity = cosine_similarity(feature_vectors)
print(similarity.shape)

(2825, 2825)


In [38]:
movie_name = input("Enter your movie name to get similar suggestions: ")

Enter your movie name to get similar suggestions: Marihuana


In [39]:
list_of_all_title = movies_data['title'].tolist()
print(list_of_all_title)

['The General', 'His Girl Friday', 'Detour', 'Marihuana', "Intolerance: Love's Struggle Throughout the Ages", 'The Stranger', 'The Phantom of the Opera', 'The Most Dangerous Game', 'Steamboat Bill, Jr.', 'The Kid', 'Cyrano de Bergerac', 'Scarlet Street', 'Made for Each Other', 'The Thief of Bagdad', 'D.O.A.', 'Jungle Book', 'Love Affair', 'Of Human Bondage', 'Our Town', 'The Little Princess', 'The Jackie Robinson Story', 'The Strange Love of Martha Ivers', "Lady Windermere's Fan", 'Meet John Doe', 'The Outlaw', 'The Front Page', 'White Zombie', 'Bird of Paradise', 'The Lost World', 'The Private Life of Henry VIII', 'The Southerner', 'A Farewell to Arms', 'The Scarlet Letter', 'The Big Lift', 'Why We Fight: Prelude to War', 'Santa Fe Trail', 'Dick Tracy', 'Woman on the Run', 'Life with Father', 'Nothing Sacred', 'Penny Serenade', 'He Walked by Night', 'Algiers', 'Jack London', 'Broken Blossoms', 'Too Late for Tears', 'Orphans of the Storm', 'Let There Be Light', 'The Red House', 'Payoff

In [40]:
finding_close_match = difflib.get_close_matches(movie_name, list_of_all_title)
print(finding_close_match)

['Marihuana', 'Sabrina']


In [41]:
close_match = finding_close_match[0]
print(close_match)

Marihuana


In [42]:
index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]
print(index_of_the_movie)

4


In [43]:
similarity_score = list(enumerate(similarity[index_of_the_movie]))
print(similarity_score)
len(similarity_score)

[(0, 0.026002282135495734), (1, 0.018463957071668265), (2, 0.012330835392393638), (3, 0.05797262030232005), (4, 1.0000000000000002), (5, 0.011495264304316587), (6, 0.025193828310711888), (7, 0.02822780795678643), (8, 0.014500450303938237), (9, 0.04029733614891493), (10, 0.03180075778370321), (11, 0.03526449630775207), (12, 0.06619690909440193), (13, 0.019958468591214275), (14, 0.027944590107530934), (15, 0.030009604465127755), (16, 0.03925671868071222), (17, 0.0382784859497569), (18, 0.010550949880139725), (19, 0.0418279848180765), (20, 0.030148661730021325), (21, 0.044646131925730165), (22, 0.06171722660181189), (23, 0.02977907988396141), (24, 0.018450462538584973), (25, 0.05137408661160208), (26, 0.05594753325043815), (27, 0.038105644321449215), (28, 0.010411261405813475), (29, 0.03528560853142921), (30, 0.015393553565964287), (31, 0.06482847152645395), (32, 0.08935198408974811), (33, 0.043220173331517175), (34, 0.02284487272440828), (35, 0.05103989023662311), (36, 0.0141769049961817

2825

In [44]:
sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True) 
print(sorted_similar_movies)

[(4, 1.0000000000000002), (2253, 0.13022198132211163), (2590, 0.11955222628151237), (786, 0.11795151695418658), (105, 0.11576822322626786), (1235, 0.11206613595495894), (634, 0.11019628456301861), (1868, 0.10711684635148466), (1957, 0.10642344219472374), (1540, 0.1056238429890829), (1004, 0.10313169250039819), (2773, 0.10205874786206712), (1754, 0.10045442959612787), (644, 0.09985397465851083), (1017, 0.09948990415220853), (1404, 0.09928252939567037), (1676, 0.09778008896220863), (2044, 0.09686203070520137), (204, 0.0958957224294373), (2083, 0.09527525706154259), (1222, 0.09442261054626772), (206, 0.09402722463069223), (534, 0.09367073700945172), (366, 0.09353106529506178), (669, 0.09345130516108936), (723, 0.09293842875628504), (221, 0.0928989670016595), (403, 0.09246386520049182), (99, 0.09210438704965955), (2035, 0.09171196819475597), (84, 0.09047388367281113), (1981, 0.09037865792209816), (1260, 0.08974905845944145), (2063, 0.08957385974696946), (1285, 0.08946232445077308), (32, 0.

In [45]:
print("These are some suggestion for your selected movie: \n")

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if (i<31):
    print(i, '.',title_from_index)
    i+=1

These are some suggestion for your selected movie: 

1 . Intolerance: Love's Struggle Throughout the Ages
2 . Epic Warrior Women
3 . A Greek Odyssey with Bettany Hughes
4 . By Love Possessed
5 . Three Came Home
6 . Mission of the Shark: The Saga of the U.S.S. Indianapolis
7 . Salt of the Earth
8 . Mothers and Daughters
9 . Cassanova Was a Woman
10 . Teen Wolf
11 . Timestalkers
12 . Murderous History
13 . Rio, I Love You
14 . The Bigamist
15 . A Matter of Sex
16 . A Mighty Heart
17 . Extant
18 . Greatest Party Story Ever
19 . Joan the Woman
20 . America's National Parks at 100
21 . Fatal Memories
22 . Indiscreet
23 . King of the Newsboys
24 . The Love of Sunya
25 . I Don't Want to Be Born
26 . A Matter of Time
27 . A Romance of the Redwoods
28 . The Lady Refuses
29 . Glorifying the American Girl
30 . The Spy in the Hanoi Hilton


In [47]:
movie_name = input("Enter your movie name to get similar suggestions: ")

list_of_all_titles = movies_data['title'].tolist()

finding_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)

close_match = finding_close_match[0]

index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]

similarity_score = list(enumerate(similarity[index_of_the_movie]))

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True) 

print("These are some suggestion for your selected movie: \n")
i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if (i<30):
    print(i, '.',title_from_index)
    i+=1

Enter your movie name to get similar suggestions: The General	
These are some suggestion for your selected movie: 

1 . His Girl Friday
2 . The Front Page
3 . By Appointment Only
4 . Good Friday
5 . East Side Kids
6 . King of the Newsboys
7 . Boy, Did I Get a Wrong Number!
8 . The Strange Love of Martha Ivers
9 . Die, Monster, Die!
10 . The Trip
11 . A Bucket of Blood
12 . 10E
13 . St. Louis Superman
14 . A Florida Enchantment
15 . Coyote
16 . Prison Break
17 . The Bet
18 . The Heavenly Kid
19 . Sold
20 . The President's Mystery
21 . Hard Promises
22 . Guilty Party
23 . City Without Men
24 . As Far As The Eye Can See
25 . Escape Clause
26 . Daddy's Home
27 . It's a Joke, Son!
28 . Another Version of You
29 . Portrait of an Escort
